In [1]:
from datetime import datetime

from binance.client import Client
from binance.enums import *

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from datetime import datetime
from binance.helpers import round_step_size

import numpy as np
import pandas as pd
import json
import time
import sys


In [2]:
#client = Client('', '')

In [3]:
#cmc = CoinMarketCapAPI('  ')

In [4]:
#info = client.get_all_tickers()

In [5]:
#quote_ccy = 'USDT'

In [6]:
#ticker_list = [d['symbol'] for d in info if d['symbol'].endswith(quote_ccy)]

In [7]:
#rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [8]:
#target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 100 and d['rank'] <= 500]

In [9]:
#target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [10]:
#target_ticker_list

In [11]:
#target_ticker_list = ['BONDUSDT']

In [12]:
global client;

In [13]:
global target_ticker_list

In [14]:
def init():
    global client
    client = Client('', '', {"verify": True, "timeout": 20})
    cmc = CoinMarketCapAPI('')
    info_all_tickers  = client.get_all_tickers()
    quote_ccy = 'USDT'
    ticker_list = [d['symbol'] for d in info_all_tickers if d['symbol'].endswith(quote_ccy)]
    rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data
    target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 10 and d['rank'] <= 50]
    global target_ticker_list
    rank_filtered_ticker_list = [d for d in ticker_list if d in target_rank_info]
    # target_ticker_list = ['RIFUSDT']
    pair_details = client.get_all_isolated_margin_symbols()
    margin_pair_list = [d['symbol'] for d in pair_details if d['quote'] == quote_ccy and d['isMarginTrade'] == True]   
    target_ticker_list = [d for d in rank_filtered_ticker_list if d in margin_pair_list]
    return target_ticker_list
    

In [15]:
def is_candidate(filtered_klines_info): 
    is_candidate = False
    
    buy_kline_info = filtered_klines_info[0]
    buy_kline_average_volume = buy_kline_info[5]

    
    strong_context_kline_info = filtered_klines_info[1:len(filtered_klines_info)]


    
    strong_context_kline_average_volume = np.mean(strong_context_kline_info[:,5])
    
    for strong_context_kline in strong_context_kline_info:
        perc_change = strong_context_kline[-1]
        perc_volume = strong_context_kline[5] / buy_kline_average_volume

        if perc_change > 5 or perc_change < -5  or perc_volume > 0.4:
            return False
        
    buy_line_open_price = buy_kline_info[1]
    buy_line_high_price = buy_kline_info[2]
    buy_line_low_price = buy_kline_info[3]
    buy_line_close_price = buy_kline_info[4]

    #Drop from high price to close price should be in the range of 10 - 15%
    drop_from_high_to_close_gate =   1.20 > buy_line_high_price/buy_line_close_price > 1.05


    #Percentage increase should be in the range of 7 - 15%
    percentage_gate = 30 > buy_kline_info[-1] > 20






    if percentage_gate and True and True:

        is_candidate = True
    else:
        is_candidate = False

    return is_candidate

In [16]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[4] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [17]:
trade_list = []
init()
test_run = False
print('Total Tickers to Analyze')
print(len(target_ticker_list))
count = 1
for target_ticker in target_ticker_list:
    
    print('Analyzing Hourly Started ')
    print(target_ticker)
    print(count)
    count = count + 1

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_1HOUR, "45 day ago UTC") 
    

    
    
    
    length = int(len(klines))
    if(length < 24):
        continue
    
    #klines.pop()
    
    

    

    array = np.array(klines, dtype=np.float64)
    focused_kline_array = np.delete(array, [6, 7, 8, 9, 10, 11], 1)
    percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
    # reduce lengrth below if poped
    percentage_value = np.reshape(percentage_value, (length, 1))
    focused_kline_array = np.append(focused_kline_array, percentage_value, 1)
    
    focused_reverse_kline_array = focused_kline_array[::-1]

    for x in range(0, length):
        hit_ticker_dictionary = {}
        
        filtered_klines_info = focused_reverse_kline_array[x:x+5]


        if(len(filtered_klines_info) == 5):


            
            if(is_candidate(filtered_klines_info)):
                    
                
                    
                buy_kline_info = filtered_klines_info[0]
                hit_ticker_dictionary = {}
                buy_price = buy_kline_info[4]
                hit_ticker_dictionary['ticker'] = target_ticker
                hit_ticker_dictionary['buy_price'] = buy_kline_info[4]

                buy_timesstamp = buy_kline_info[0]
                hit_ticker_dictionary['buy_datetime'] = str(datetime.fromtimestamp(buy_timesstamp // 1000))
                
                proft_percentage = 1.35
                loss_percentage = 0.8
                profit_sell_price = buy_price * proft_percentage
                loss_sell_price = buy_price * loss_percentage
                
                
                
                usdt = 1000
                
                buy_quantity = usdt/buy_price
                hit_ticker_dictionary['quantity'] = buy_quantity
                
                
 
                
                for y in range(0, x) :
                    
                    start = x - (y + 1)
                    end = x - y
                    
                    high_price = focused_reverse_kline_array[start:end][0][2]
                    low_price = focused_reverse_kline_array[start:end][0][3]
                    close_price = focused_reverse_kline_array[start:end][0][4]
                    sell_timesstamp = focused_reverse_kline_array[start:end][0][0]
                    sell_datetime = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                    
                    if high_price >= profit_sell_price:
                        hit_ticker_dictionary['sell_price'] = profit_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_reverse_kline_array[start:end][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = profit_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * proft_percentage - usdt
                        break
                    elif sell_timesstamp - buy_timesstamp >= 6600000 :
                        hit_ticker_dictionary['sell_price'] = close_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_reverse_kline_array[start:end][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = close_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = buy_quantity * close_price - buy_quantity * buy_price
                        break   
                    elif False: # low_price <=  loss_sell_price:
                        hit_ticker_dictionary['sell_price'] = loss_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_reverse_kline_array[start:end][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = loss_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * loss_percentage - usdt
                        break
                        
                        
                trade_list.append(hit_ticker_dictionary)        


Total Tickers to Analyze
35
Analyzing Hourly Started 
LTCUSDT
1
Analyzing Hourly Started 
EOSUSDT
2
Analyzing Hourly Started 
TUSDUSDT
3
Analyzing Hourly Started 
XLMUSDT
4
Analyzing Hourly Started 
TRXUSDT
5
Analyzing Hourly Started 
ETCUSDT
6
Analyzing Hourly Started 
VETUSDT
7
Analyzing Hourly Started 
LINKUSDT
8
Analyzing Hourly Started 
XMRUSDT
9
Analyzing Hourly Started 
ZECUSDT
10
Analyzing Hourly Started 
THETAUSDT
11
Analyzing Hourly Started 
MATICUSDT
12
Analyzing Hourly Started 
ATOMUSDT
13
Analyzing Hourly Started 
ALGOUSDT
14
Analyzing Hourly Started 
DOGEUSDT
15
Analyzing Hourly Started 
XTZUSDT
16
Analyzing Hourly Started 
HBARUSDT
17
Analyzing Hourly Started 
BCHUSDT
18
Analyzing Hourly Started 
FTTUSDT
19
Analyzing Hourly Started 
MANAUSDT
20
Analyzing Hourly Started 
SANDUSDT
21
Analyzing Hourly Started 
DOTUSDT
22
Analyzing Hourly Started 
EGLDUSDT
23
Analyzing Hourly Started 
UNIUSDT
24
Analyzing Hourly Started 
AVAXUSDT
25
Analyzing Hourly Started 
HNTUSDT
26
Analy

In [18]:
trade_list

[{'ticker': 'FLOWUSDT',
  'buy_price': 2.316,
  'buy_datetime': '2022-08-04 16:00:00',
  'quantity': 431.77892918825563,
  'sell_price': 2.58,
  'sell_datetime': '2022-08-04 18:00:00',
  'days_to_gain_loss': 0,
  'price_diff': 0.26400000000000023,
  'proft_loss': 113.98963730569949}]

In [19]:
a = 10
b = 5

step = -1 if a > b else 1
for x in range (a, b, step):
    print(x, end = ' ')

10 9 8 7 6 

In [20]:
len(trade_list)

1

In [21]:
df = pd.DataFrame.from_dict(trade_list)

In [22]:
print('Profit and Loss')
df['proft_loss'].sum()

Profit and Loss


113.98963730569949

In [23]:
print('Number of Profit Trades')
df[df['price_diff'] > 0].shape[0]

Number of Profit Trades


1

In [24]:
print('Number of Loss Trades')
df[df['price_diff'] < 0].shape[0]

Number of Loss Trades


0

In [25]:
with pd.option_context('display.max_rows', None):
    display(df.sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
0,FLOWUSDT,2.316,2022-08-04 16:00:00,431.778929,2.58,2022-08-04 18:00:00,0,0.264,113.989637


In [26]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] > 0].sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
0,FLOWUSDT,2.316,2022-08-04 16:00:00,431.778929,2.58,2022-08-04 18:00:00,0,0.264,113.989637


In [27]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] < 0].sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss


In [28]:
with pd.option_context('display.max_rows', None):
    display(df[pd.isna(df['sell_price'])].sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss


In [29]:
with pd.option_context('display.max_rows', None):
    display(df[np.invert(pd.isna(df['sell_price']))].sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
0,FLOWUSDT,2.316,2022-08-04 16:00:00,431.778929,2.58,2022-08-04 18:00:00,0,0.264,113.989637


In [30]:
info = client.get_symbol_info('FLMUSDT')
print(info['filters'])

[{'filterType': 'PRICE_FILTER', 'minPrice': '0.00010000', 'maxPrice': '1000.00000000', 'tickSize': '0.00010000'}, {'filterType': 'PERCENT_PRICE', 'multiplierUp': '5', 'multiplierDown': '0.2', 'avgPriceMins': 5}, {'filterType': 'LOT_SIZE', 'minQty': '1.00000000', 'maxQty': '900000.00000000', 'stepSize': '1.00000000'}, {'filterType': 'MIN_NOTIONAL', 'minNotional': '10.00000000', 'applyToMarket': True, 'avgPriceMins': 5}, {'filterType': 'ICEBERG_PARTS', 'limit': 10}, {'filterType': 'MARKET_LOT_SIZE', 'minQty': '0.00000000', 'maxQty': '1319206.31115731', 'stepSize': '0.00000000'}, {'filterType': 'TRAILING_DELTA', 'minTrailingAboveDelta': 10, 'maxTrailingAboveDelta': 2000, 'minTrailingBelowDelta': 10, 'maxTrailingBelowDelta': 2000}, {'filterType': 'MAX_NUM_ORDERS', 'maxNumOrders': 200}, {'filterType': 'MAX_NUM_ALGO_ORDERS', 'maxNumAlgoOrders': 5}]


In [31]:
print(info['filters'][0]['tickSize'])

0.00010000


In [32]:
symbol_info = {}
def get_symbol_info(symbols):
    global symbol_info
    for symbol in symbols:
        info = client.get_symbol_info(symbol)
        base_asset = info['baseAsset']
#  PRICE_FILTER
        price_filter_min_quantity = float(info['filters'][0]['minPrice'])
        price_filter_max_quantity = float(info['filters'][0]['maxPrice'])
        price_filter_tick_size = float(info['filters'][0]['tickSize'])
        
#  LOT_SIZE (for market- and limit orders)
        lot_size_min_quantity = float(info['filters'][2]['minQty'])
        lot_size_max_quantity = float(info['filters'][2]['maxQty'])
        lot_size_step_size = float(info['filters'][2]['stepSize'])
        
#  MIN NOTIONAL
        min_notional = float(info['filters'][3]['minNotional'])
        symbol_info[symbol] = {'price_filter_min_quantity': price_filter_min_quantity, 'price_filter_max_quantity': price_filter_max_quantity,\
                               'price_filter_tick_size': price_filter_tick_size,'lot_size_min_quantity': lot_size_min_quantity, \
                                'lot_size_min_quantity': lot_size_min_quantity, 'lot_size_max_quantity': lot_size_max_quantity, \
                               'lot_size_step_size': lot_size_step_size, 'min_notional': min_notional, }
        


In [33]:
info = client.get_margin_asset(asset='BNB')

In [34]:
info

{'assetName': 'BNB',
 'assetFullName': 'Binance Coin',
 'isBorrowable': True,
 'isMortgageable': True,
 'userMinBorrow': '0',
 'userMinRepay': '0'}

In [35]:
info = client.get_margin_symbol(symbol='BURGERUSDT')

In [36]:
pair_details = client.get_all_isolated_margin_symbols()

In [37]:
info = client.get_isolated_margin_symbol(symbol='BURGERUSDT')

In [38]:
info = client.get_all_isolated_margin_symbols()

In [39]:
info = client.get_margin_price_index(symbol='MOVRUSDT')

In [40]:
info

{'symbol': 'MOVRUSDT', 'price': '17.20556923', 'calcTime': 1660296462000}

In [41]:
#transaction = client.transfer_spot_to_margin(asset='USDT', amount='100')

In [42]:
#transaction = client.create_margin_loan(asset='USDT', amount='200')

In [43]:
#transaction

In [44]:
#details = client.get_margin_loan_details(asset='USDT', txId='114199991053')

In [45]:
#transaction = client.repay_margin_loan(asset='USDT', amount='200')

In [46]:
#transaction

In [47]:
#details = client.get_margin_loan_details(asset='USDT', txId='114199991053')

In [48]:
#details

In [49]:
#order = client.create_margin_order(symbol='MOVRUSDT', side=SIDE_SELL, type='MARKET', quoteOrderQty=200, isIsolated='TRUE')

In [50]:
#transaction = client.transfer_spot_to_isolated_margin(asset='USDT', symbol='MOVRUSDT', amount='100')

In [51]:
#transaction

In [52]:
#transaction = client.create_margin_loan(asset='USDT', amount='200', symbol='MOVRUSDT', isIsolated='TRUE')

In [53]:
#transaction = client.repay_margin_loan(asset='USDT', amount='200', symbol='MOVRUSDT', isIsolated='TRUE')

In [54]:
#transaction = client.create_margin_loan(asset='MOVR', amount='200', symbol='MOVRUSDT', isIsolated='TRUE')

In [55]:
#transaction = client.transfer_isolated_margin_to_spot(asset='USDT', symbol='MOVRUSDT', amount='99.99833334')

In [56]:
#details = client.get_max_margin_transfer

In [57]:
#details

In [58]:
orders = client.get_all_margin_orders(symbol='BURGERUSDT', limit=10)

In [59]:
orders

[]

In [60]:
info = client.get_margin_account()

In [61]:
info

{'tradeEnabled': True,
 'transferEnabled': True,
 'borrowEnabled': True,
 'marginLevel': '999',
 'totalAssetOfBtc': '0.00417331',
 'totalLiabilityOfBtc': '0.00000007',
 'totalNetAssetOfBtc': '0.00417325',
 'userAssets': [{'asset': 'AGLD',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'MATIC',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'BURGER',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'STPT',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'CTXC',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'IOTX',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'TVK',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': 

In [62]:
#transaction = client.transfer_margin_to_spot(asset='BTC', amount='0.00430996')

In [63]:
transaction

NameError: name 'transaction' is not defined